In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [14]:
class prepare_stockdata: 
    def getstockdata(self,path):
        df = pd.read_csv(path)
        df['Date'] = pd.to_datetime(df['Date'])
        df.sort_values(by=['Date'], inplace=True)
        df.index = df["Date"]
        df = df.drop(['Date'], axis = 1)
        return df

    def getstockfeatures(self,df,features):
        df1 = pd.DataFrame(df[features])
        return df1

    def add_features(self,df):
        df['7DayAve'] = df['Close'].rolling(7).mean()
        df['15DayAve'] = df['Close'].rolling(15).mean()
        df["Day_Perc_Change"] = df["Close"].pct_change()*100
        df.dropna(axis = 0, inplace = True)
        return df

    #days_ahead can be set of -1 for next day prediction
    #df is the dataframe containing features
    def set_predictioncol(self,df,days_ahead):
        df['Prediction'] = df[['Close']].shift(days_ahead)
        return df

    def prepare_dataset(self,df,days_ahead):
        #Create a data set X and convert it into numpy array , which will be having actual values
        X = np.array(df.drop(['Prediction'],1))
        #Remove the last N rows

        X = X[:days_ahead]

        #print('X : \n',X)
        # Create a dataset y which will be having Predicted values and convert into numpy array
        y = np.array(df['Prediction'])
        # Remove Last 15 rows
        y = y[:days_ahead]
        #print(y)
        return X, y
    
    def preparation_driver(self,path,features,days_ahead):
        

        #Step 1 : Load Data
        df = self.getstockdata(path)

        #Step 2: Select relevant Features
        df = self.getstockfeatures(df,features)

        #Step 3: Add relevant features
        df = self.add_features(df)

        #Step 4: Setup/Create prediction column
        df = self.set_predictioncol(df,days_ahead)

        ##Step 5: Prepare dataset for classification
        X, y = self.prepare_dataset(df,days_ahead)

        #Step 6: Split the data into train and test with 90 & 10 % respectively
        x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

        #To return X part of dataframe without movemnet
        X_df = np.array(df.drop(['Prediction'],1))

        return df,x_train, x_test, y_train, y_test,X_df




In [15]:
path = "D:\\Stock Data\\TRG.csv"
features = ['LDCP','Close']
days_ahead = -1
prp = prepare_stockdata()
df,x_train, x_test, y_train, y_test,X_df = prp.preparation_driver(path,features,days_ahead)
df.tail(10)


C:\Users\Hassan\AppData\Local\Temp/ipykernel_7036/3864821332.py:29: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X = np.array(df.drop(['Prediction'],1))
C:\Users\Hassan\AppData\Local\Temp/ipykernel_7036/3864821332.py:64: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X_df = np.array(df.drop(['Prediction'],1))


,LDCP,Close,7DayAve,15DayAve,Day_Perc_Change,Prediction
Date,,,,,,
2021-12-15,96.47,103.70,94.612857,89.654667,7.494558,107.82
2021-12-16,103.70,107.82,97.561429,90.922667,3.972999,106.43
2021-12-17,107.82,106.43,99.378571,92.189333,-1.289186,114.41
2021-12-20,106.43,114.41,102.144286,93.808000,7.497886,116.24
2021-12-21,114.41,116.24,104.972857,95.748667,1.599511,117.53
2021-12-22,116.24,117.53,108.942857,97.694000,1.109773,124.35
2021-12-23,117.53,124.35,112.925714,100.535333,5.802774,118.40
2021-12-24,124.35,118.40,115.025714,103.238000,-4.784881,112.97
2021-12-27,118.40,112.97,115.761429,105.362667,-4.586149,106.00
